
# Función para leer el set de datos



In [1]:
import numpy as np
import os
import gzip

def cargar_set(rute, tipo="train"):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz'%tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype = np.uint8, offset = 8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype = np.uint8, offset= 16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

# Acceso a la información en Google Drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = 'gdrive/My Drive/Big_Data_unal/Data_sets/fashion_mnist_data'

X_train, Y_train = cargar_set(ruta, tipo = 'train')
X_test, Y_test = cargar_set(ruta, tipo = 'test')

Mounted at /content/gdrive



#Reajustar (Reshape) del tamaño de los datos


In [3]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train, (59904,28,28,1))
X_test = np.reshape(X_test, (9984,28,28,1))

# Importar TensorFlow 2 con Keras

In [4]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de TensorFlow' + tf.__version__)

Versión de TensorFlow2.4.1


# Creación del modelo


In [5]:
tf.random.set_seed(200)

modelo = tf.keras.models.Sequential()

modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding = 'same', activation ='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding = 'same', activation ='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding = 'same', activation ='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.Flatten())
modelo.add(tf.keras.layers.Dense(256))
modelo.add(tf.keras.layers.Activation('elu'))
modelo.add(tf.keras.layers.Dropout(0.5))
modelo.add(tf.keras.layers.Dense(10))
modelo.add(tf.keras.layers.Activation('softmax'))

# Imprimir en pantalla la información del modelo

modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0

# Verificación de la disponibilidad de la GPU

In [6]:
nombre_gpu = tf.test.gpu_device_name()

if nombre_gpu != '/device:GPU:0':
  raise SystemError( 'GPU no encontrada')

print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


# Entrenamiento con CPU

In [7]:
modelo.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [8]:
import timeit


def entrenamiento_gpu():
  
  with tf.device('/device:GPU:0'):
    modelo.fit(X_train, Y_train, validation_data = (X_test, Y_test), batch_size = 128, epochs = 2, verbose = 1)

    return None


gpu_time = timeit.timeit('entrenamiento_gpu()', number = 1, setup = 'from __main__ import entrenamiento_gpu')

Epoch 1/2
468/468 [==============================] - 18s 23ms/step - loss: 1.0180 - accuracy: 0.7072 - val_loss: 0.3720 - val_accuracy: 0.8718
Epoch 2/2
468/468 [==============================] - 10s 22ms/step - loss: 0.4089 - accuracy: 0.8548 - val_loss: 0.3405 - val_accuracy: 0.8832


In [9]:
print('El tiempo de entrenamiento en GPU es de: ' + str(gpu_time) + " segundos")

El tiempo de entrenamiento en GPU es de: 28.067428245999963 segundos
